In [1]:
# 코랩에 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import warnings
# warnings.filterwarnings(action = "ignore")

# 데이터 처리 모듈
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import time
from tqdm import tqdm
pd.options.display.float_format = "{:.5f}".format
# pd.reset_option("display.float_format")

# 멀티프로세싱
import multiprocessing as mp
from multiprocessing import Pool

# 척도 변환 모듈(표준화, 정규화, 로버스트 정규화, 원핫인코딩(범주형에서 더미변수로 변환) 등)
from sklearn.preprocessing import *

# 결측값 관측 모듈
import missingno as msno

# 시각화 모듈
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

# 배열, 행렬 연산 모듈
import numpy as np

# 데이터 분할 모듈(훈련용 / 검증용 / 시험용)
from sklearn.model_selection import train_test_split

# 통계적 가설검정 및 계량화 모듈
# https://youtu.be/FtWEZw3kUho
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from scipy.interpolate import UnivariateSpline

# 타입 어노테이션 모듈
from typing import *

# 시간 관련 모듈
from tqdm import tqdm
from time import strptime, sleep

# 웹크롤링 / 스크래핑 관련 모듈
import requests
import io
import zipfile
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET

# 결측값 대체 및 특징 추출
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA, SparsePCA

# 연관규칙 분석 실행 모듈
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx

# 한글폰트 설정
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
import matplotlib.patches as mpatches

Mounted at /content/drive


**6개의 투자심리 측정변수**
* NYSE의 거래회전율
* 배당금 프리미엄
* 폐쇄형 펀드의 할인
율
* IPO의 첫 거래일 평균 수익률
* 채권 대비 주식발행 비중
* 투자심리선은 해당 종목이 과매수 또는 과매도 상태인지를 파악함으로써 투자자 심리가 낙관적인지 비관적인지를 판단하는데 사용하며, 일반적으로 75% 이상이면 해당 주식이 과매수 상태, 25% 이하이면 과매도 상태로 해석

**6개 변수를 이용하여 월별 투자 심리지수를 도출**
* ```개인투자자의 매수매도 불균형(BSI, buying and selling imbalance of individual investors)```

* ```주식형 펀드 유입액(FUND, stock fund flow ratio)```

  예금이나 채권형 펀드에서 위험한 주식형 펀드로 자금이 이동하는 현상

* ```소비자 기대지수(CEI, customer expectation index)```
  
  통계청에서 발간하는 소비자 기대지수는 경기 순환을 예측하는 지수로 본 연구에서는 자연로그를 취하여 사용(경기선행지수 중 1)

  (i) 현재생활형편, 생활형편전망, 가계수입전망, 소비지출전망, 현재경기판단, 향후경기전망 6개의 개별지수의 합산지수 산출

  (ii) 표준화하고, 6개 구성지수 표준편차의 중간값을 가중치로 곱하여 소비자 심리지수 산출

  (iii) 100보다 높으면 주관적 기대심리 과거 평균 대비 낙관적으로 해석(기준치 상회 여부 및 지수 자체의 증감 중요)

* ```주식투자 예치금(CD, customer’s deposit for stock investment)```
  
  주식투자를 위해 예치한 금액이 얼마나 변동하였는지를 측정하여 추가적으로 고려한다. 주식투자 예치금은 주식을 매수하기 위한 일시적인 예치금으로 투자자들이 주식시장에 대해 보다 낙관적인 견해를 가지고 있을수록 증가

* ```상장주식 거래회전율(TURN, turnover ratio)```

* ```주식 자금조달 비율(SR, equity shares issued ratio)```

  주식과 채권을 합계한 총 자금 조달금액에서 주식발행을 통해 조달한 자금의 비중을 사용

* 투자심리지수 파생컬럼

0) **10세 기준 연령대, 관리사원 존재여부**

1) **전환율(CTR)**

    매수전환율 = 매수체결횟수 / 접속일수
    
    매도전환율 = 매도체결횟수 / 접속일수

* CONN_DYS < MTS_DYS + HTS_DYS
* 특정기간동안의 HTS 또는 MTS 접속일수

2) **최고자산 대비 등락률**

    {(월말일 기준 예수금 등 현금성 자산액 + 월말일 기준 국내주식 자산 평가액 + 월말일 기준 해외주식 자산 평가액) - (과거 최고자산액)} / (과거 최고자산액) * 100

    특이 시점 - 과거 최고자산액 기록 연월

3) **거래회전율 = 1월부터 3월까지 총 거래량 / 특정기간동안의 거래량**
  
    주식매수대금회전율 = 3개월간 주식 매수액 합계 / 특정기간동안의 주식 매수액 합계

    주식매도대금회전율 = 3개월간 주식 매도액 합계 / 특정기간동안의 주식 매도액 합계
    
    주식매수량회전율 = 3개월간 주식 매수종목수 합계 / 특정기간동안의 주식 매수종목수 합계
    
    주식매도량회전율 = 3개월간 주식 매도종목수 합계 / 특정기간동안의 주식 매도종목수 합계

4) **주식수익률**

    국내주식수익률 = 월말일 기준 국내주식 자산 평가액 / 월말일 기준 국내주식 자산 매입액
    해외주식수익률 = 월말일 기준 국내주식 자산 평가액 / 월말일 기준 국내주식 자산 매입액


5) **매수매도 불균형(IBSI)**

    매수매도불균형 = 특정기간동안의 주식 매수액 합계 / 특정기간동안의 주식 매도액 합계

6) **대체입출고비중[타 증권사 주식이체]  >= 50%**
    
    유가증권대체입고[진입율] = (특정기간동안의 주식 입고액 합계 + 특정기간동안의 입금액 합계) / (월말일 기준 예수금 등 현금성 자산액 + 월말일 기준 국내주식 자산 평가액)

    유가증권대체출고[이탈률] = (특정기간동안의 주식 출고액 합계 + 특정기간동안의 출금액 합계) / (월말일 기준 예수금 등 현금성 자산액 + 월말일 기준 국내주식 자산 평가액)
    
7) **분산투자지표**

    (특정기간동안의 주식매수액 1위 종목매수액 합계 + 특정기간동안의 주식매수액 2위 종목매수액 합계 + 특정기간동안의 주식매수액 3위 종목매수액 합계) /  특정기간동안의 주식 매수액 합계

    (특정기간동안의 주식매도액 1위 종목매도액 합계 + 특정기간동안의 주식매도액 2위 종목매도액 합계 + 특정기간동안의 주식매도액 3위 종목매도액 합계) /  특정기간동안의 주식 매도액 합계


8) **이전6개월 매도액 중 특정기간 이내의 매수매도발생 매도액비중** (PCA)


* 나를 위한 것만은 아닌 기록

```
CS_DATA_M2_3_BUY1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_BUY1.csv", index_col = False).fillna(0)
CS_DATA_M2_3_BUY2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_BUY2.csv", index_col = False).fillna(0)
CS_DATA_M2_3_BUY3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_BUY3.csv", index_col = False).fillna(0)
CS_DATA_M2_3_SELL1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_SELL1.csv", index_col = False).fillna(0)
CS_DATA_M2_3_SELL2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_SELL2.csv", index_col = False).fillna(0)
CS_DATA_M2_3_SELL3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_SELL3.csv", index_col = False).fillna(0)
```

```
CS_DATA_M3_1_BUY1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_BUY1.csv", index_col = False).fillna(0)
CS_DATA_M3_1_BUY2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_BUY2.csv", index_col = False).fillna(0)
CS_DATA_M3_1_BUY3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_BUY3.csv", index_col = False).fillna(0)
CS_DATA_M3_1_SELL1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_SELL1.csv", index_col = False).fillna(0)
CS_DATA_M3_1_SELL2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_SELL2.csv", index_col = False).fillna(0)
CS_DATA_M3_1_SELL3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_SELL3.csv", index_col = False).fillna(0)
```

```
CS_DATA_M4_2_BUY1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_BUY1.csv", index_col = False).fillna(0)
CS_DATA_M4_2_BUY2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_BUY2.csv", index_col = False).fillna(0)
CS_DATA_M4_2_BUY3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_BUY3.csv", index_col = False).fillna(0)
CS_DATA_M4_2_SELL1 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_SELL1.csv", index_col = False).fillna(0)
CS_DATA_M4_2_SELL2 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_SELL2.csv", index_col = False).fillna(0)
CS_DATA_M4_2_SELL3 = pd.read_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_SELL3.csv", index_col = False).fillna(0)
```

```
CONSUMER_ID = pd.merge(
    CS_DATA_M2_3["고객번호"], CS_DATA_M3_1["고객번호"], how = "inner")
CONSUMER_ID = pd.merge(
    CS_DATA_M4_2["고객번호"], CONSUMER_ID, how = "inner")

for dataframe in [CS_DATA_M2_3, CS_DATA_M3_1, CS_DATA_M4_2]:
    # 150446 | 194670 | 168290
  print(len(dataframe))
len(CONSUMER_ID)
  # 91024
```

* 1,2,3위 평가액 또는 매입액 합계 > 전체 평가액 또는 매입액보다 큰 경우는 익명처리 과정에서 다른 값으로 대체되거나 금액이 특정 자리 이하로 절삭됨에  따라 발생한 케이스로, 해당 내용 분석에 참고하시기 바랍니다.

In [148]:
class MutateColumn():

  def __init__(self, M2_3_file_path: str, M3_1_file_path: str, M4_2_file_path: str, CS_DATA_MONTH_DATA = None):

    self.CS_DATA_M2_3_BUY1 = pd.read_csv(M2_3_file_path, index_col = False).fillna(0)
    self.CS_DATA_M3_1_BUY1 = pd.read_csv(M3_1_file_path, index_col = False).fillna(0)
    self.CS_DATA_M4_2_BUY1 = pd.read_csv(M4_2_file_path, index_col = False).fillna(0)
    self.CS_DATA_MONTH_DATA = CS_DATA_MONTH_DATA

  def readDataset(self):
    return self.CS_DATA_M2_3_BUY1, self.CS_DATA_M3_1_BUY1, self.CS_DATA_M4_2_BUY1


  def periodDatetime(self):
    for basedate, dataframe in zip(["202301", "202302", "202303"], [self.CS_DATA_M2_3_BUY1, self.CS_DATA_M3_1_BUY1, self.CS_DATA_M4_2_BUY1]):
      dataframe["과거 최고자산액 기록 연월"] = dataframe["과거 최고자산액 기록 연월"].apply(
          lambda data: datetime.datetime.strptime(str(data), "%Y%m"))
      dataframe["과거 최고자산액 기록 연월"] = ( datetime.datetime.strptime(basedate, "%Y%m") - dataframe["과거 최고자산액 기록 연월"] )
      # [timedelta] 두 날짜 또는 시간의 차이가 어느정도인지 나타낼 때 사용하는 모듈
      dataframe["과거 최고자산액 기록 연월"] = dataframe["과거 최고자산액 기록 연월"].dt.days
      return self.CS_DATA_M2_3_BUY1, self.CS_DATA_M3_1_BUY1, self.CS_DATA_M4_2_BUY1


  def linearCombinationPCA(self, dataset: pd.DataFrame, k: int):
  # 1. 표준화
    # mean_matrix = dataset.mean()
    # std_matrix = dataset.std()
    # substract_mean_matrix = (dataset - mean_matrix)

  # 2. 공분산 행렬(Calculate the Covariance Matrix of the mean-centered data.)
    covariance_matrix = np.cov(dataset.T)
    eigen_values, eigen_vectors = np.linalg.eigh(covariance_matrix)

  # 3. 고윳값과 고유벡터를 내림차순 정렬(모든 고유벡터는 서로 직교함)
    sorted_index = np.arange(0, len(eigen_values), 1)
    sorted_index = ([x for _, x
                     in sorted(zip(eigen_values, sorted_index))])[::-1]

    sorted_eigen_values = eigen_values[sorted_index]
    sorted_eigen_vectors = eigen_vectors[:, sorted_index]
    # print("내림차순 정렬한 고유값",  "\n", sorted_eigen_values, "\n", "내림차순 정렬한 고유벡터",  "\n", sorted_eigen_vectors)

  # 4. 원하는 결합정도에 따라 상위 k의 고유벡터를 선택하는데 모든 고유값의 누적합계로  설명된 분산 기여율 계산
    sum_eigen_values = np.sum(sorted_eigen_values)
    explained_variance = sorted_eigen_values / sum_eigen_values
    cummulative_variance = np.cumsum(explained_variance)
    # print("누적 분산 기여율", "\n", cummulative_variance, "\n")

  # 5. 변환된 데이터로 고유벡터의 전치행렬의 내적을 취하여 원본 데이터를 결합(데이터를 중앙에 배치하기 위해 평균을 뺌)
    n_components = k
      # k = 1
    eigen_vectors_subset = sorted_eigen_vectors[:, 0:n_components]

  # 6. 고유벡터를 사용하여 데이터의 좌표방향변환(회전) -> 정사영 / 투영 / 내적
    pca_matrix = np.dot(dataset, eigen_vectors_subset)
    # print("좌표변환한 행렬", "\n", pca_matrix, "\n")

    reduction_matrix = pd.Series(np.array(pca_matrix, ndmin = 1).tolist())
    for parenthesis in ["[", "]"]:
      reduction_matrix = reduction_matrix.apply(lambda data: str(data).replace(parenthesis, ""))
    reduction_matrix = reduction_matrix.astype("float")
    # reduction_matrix.index = dataset.index
    return abs(reduction_matrix)


  def mutateInvestorSentimentIndex(self, dataframe):
  # CS_DATA_MONTH_LIST: List = []
    CS_DATA_MONTH_DATA = pd.DataFrame()

  # 고객번호 | 연령대 | 관리사원 존재여부
    CS_DATA_MONTH_DATA["고객번호"] = dataframe["고객번호"]
    CS_DATA_MONTH_DATA["10세기준_연령대"] = dataframe["10세 기준 연령대"]
    CS_DATA_MONTH_DATA["관리사원_존재여부"] = dataframe["관리사원 존재여부"].apply(lambda data: 1 if data == "Y" else 0)

  # 전환율
    CS_DATA_MONTH_DATA["매수전환율"] = dataframe["특정기간동안의 주식 매수체결횟수"] / dataframe["특정기간동안의 HTS 또는 MTS 접속일수"]
    CS_DATA_MONTH_DATA["매도전환율"] = dataframe["특정기간동안의 주식 매도체결횟수"] / dataframe["특정기간동안의 HTS 또는 MTS 접속일수"]

  # 최고자산 대비 등락률 및 변동기간
    CS_DATA_MONTH_DATA["최고자산_대비_등락률"] = (
        ((dataframe["월말일 기준 예수금 등 현금성 자산액"] + dataframe["월말일 기준 국내주식 자산 평가액"] + dataframe["월말일 기준 해외주식 자산 평가액"]) - dataframe["과거 최고자산액"]) / dataframe["과거 최고자산액"]) * 100
    CS_DATA_MONTH_DATA["최고자산_대비_변동기간"] = dataframe["과거 최고자산액 기록 연월"]

  # 거래회전율 = 1월부터 3월까지 총 거래량 / 특정기간동안의 거래량
    CS_DATA_MONTH_DATA["주식매수대금회전율"] = (dataframe["3개월간 주식 매수액 합계"] / dataframe["특정기간동안의 주식 매수액 합계"])
    CS_DATA_MONTH_DATA["주식매도대금회전율"] = (dataframe["3개월간 주식 매도액 합계"] / dataframe["특정기간동안의 주식 매도액 합계"])
    CS_DATA_MONTH_DATA["주식매수회전율"] = (dataframe["3개월간 주식 매수종목수 합계"] / dataframe["특정기간동안의 주식 매수종목수"])
    CS_DATA_MONTH_DATA["주식매도회전율"] = (dataframe["3개월간 주식 매도종목수 합계"] / dataframe["특정기간동안의 주식 매도종목수"])

  # 수익률
    CS_DATA_MONTH_DATA["국내주식수익률"] = (dataframe["월말일 기준 국내주식 자산 평가액"] / dataframe["월말일 기준 국내주식 자산 매입액"])
    CS_DATA_MONTH_DATA["해외주식수익률"] = (dataframe["월말일 기준 해외주식 자산 평가액"] / dataframe["월말일 기준 해외주식 자산 매입액"])

  # 매수매도 불균형
    CS_DATA_MONTH_DATA["매수매도불균형"] = (dataframe["특정기간동안의 주식 매수액 합계"] / dataframe["특정기간동안의 주식 매도액 합계"])

  # 대체입출고비중
    CS_DATA_MONTH_DATA["유가증권대체입고비중[진입율]"] = ( (dataframe["특정기간동안의 주식 입고액 합계"] + dataframe["특정기간동안의 입금액 합계"]) / (dataframe["월말일 기준 예수금 등 현금성 자산액"] + dataframe["월말일 기준 국내주식 자산 평가액"]) )
    CS_DATA_MONTH_DATA["유가증권대체출고비중[이탈률]"] = ( (dataframe["특정기간동안의 주식 출고액 합계"] + dataframe["특정기간동안의 출금액 합계"]) / (dataframe["월말일 기준 예수금 등 현금성 자산액"] + dataframe["월말일 기준 국내주식 자산 평가액"]) )

  # 분산투자지표
    CS_DATA_MONTH_DATA["분산투자지표_매수기준"] = ( (dataframe["특정기간동안의 주식매수액 1위 종목매수액 합계"] + dataframe["특정기간동안의 주식매수액 2위 종목매수액 합계"] + dataframe["특정기간동안의 주식매수액 3위 종목매수액 합계"]) / (dataframe["특정기간동안의 주식 매수액 합계"]) )
    CS_DATA_MONTH_DATA["분산투자지표_매도기준"] = ( (dataframe["특정기간동안의 주식매도액 1위 종목매수액 합계"] + dataframe["특정기간동안의 주식매도액 2위 종목매수액 합계"] + dataframe["특정기간동안의 주식매도액 3위 종목매수액 합계"]) / (dataframe["특정기간동안의 주식 매도액 합계"]) )

    CS_DATA_MONTH_DATA["이전6개월_매도액중_특정기간이내의_매수후_매도액비중"] = self.linearCombinationPCA(
        dataset = dataframe[["이전6개월 매도액 중 당일매수매도발생 매도액비중",
                             "이전6개월 매도액 중 매수 후 5일 이내 매도액비중",
                             "이전6개월 매도액 중 매수 후 30일 이내 매도액비중",
                             "이전6개월 매도액 중 매수 후 180일 이내 매도액비중",
                             "이전6개월 매도액 중 매수 후 1년 이내 매도액비중",
                             "이전6개월 매도액 중 매수 후 2년 이후 이내 매도액비중"]], k = 1)
  # CS_DATA_MONTH_LIST.append(CS_DATA_MONTH_DATA)
    return CS_DATA_MONTH_DATA


  def loadDataset(self):
    self.CS_DATA_M2_3 = self.mutateInvestorSentimentIndex(self.CS_DATA_M2_3_BUY1).fillna(0)
    self.CS_DATA_M3_1 = self.mutateInvestorSentimentIndex(self.CS_DATA_M3_1_BUY1).fillna(0)
    self.CS_DATA_M4_2 = self.mutateInvestorSentimentIndex(self.CS_DATA_M4_2_BUY1).fillna(0)

    return self.CS_DATA_M2_3, self.CS_DATA_M3_1, self.CS_DATA_M4_2



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [149]:
ClusterInstance = MutateColumn(
  M2_3_file_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M2_3_BUY1.csv",
  M3_1_file_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M3_1_BUY1.csv",
  M4_2_file_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/CS_DATA_M4_2_BUY1.csv"
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [150]:
BEFORE_DATA_M2_3, BEFORE_DATA_M3_1, BEFORE_DATA_M4_2 = ClusterInstance.readDataset()
RAW_DATA_M2_3, RAW_DATA_M3_1, RAW_DATA_M4_2 = ClusterInstance.periodDatetime()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [151]:
CS_DATA_M2_3, CS_DATA_M3_1, CS_DATA_M4_2 = ClusterInstance.loadDataset()
  # CS_DATA_M2_3 = ClusterInstance.mutateInvestorSentimentIndex(dataframe = RAW_DATA_M2_3)
  # CS_DATA_M3_1 = ClusterInstance.mutateInvestorSentimentIndex(dataframe = RAW_DATA_M3_1)
  # CS_DATA_M4_2 = ClusterInstance.mutateInvestorSentimentIndex(dataframe = RAW_DATA_M4_2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [152]:
display(CS_DATA_M2_3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,고객번호,10세기준_연령대,관리사원_존재여부,매수전환율,매도전환율,최고자산_대비_등락률,최고자산_대비_변동기간,주식매수대금회전율,주식매도대금회전율,주식매수회전율,주식매도회전율,국내주식수익률,해외주식수익률,매수매도불균형,유가증권대체입고비중[진입율],유가증권대체출고비중[이탈률],분산투자지표_매수기준,분산투자지표_매도기준,이전6개월_매도액중_특정기간이내의_매수후_매도액비중
0,2,40,0,1.81818,1.54545,-16.81739,214,2.82626,2.79020,770.94118,6.29412,0.60784,0.56000,0.98039,0.00000,0.00000,0.21800,0.21373,0.32983
1,3,40,1,inf,inf,-76.66667,365,32.62000,17.14667,390000007.50000,7.50000,0.00000,0.00000,0.66667,0.00000,0.00000,1.05000,0.99333,0.38359
2,5,50,0,0.66667,0.33333,-60.30952,640,220.97872,90.91892,660000018.00000,19.00000,0.55333,0.00000,1.27027,0.02160,0.00600,1.00000,1.00000,0.37143
3,6,40,0,0.14286,0.57143,-83.84615,852,59.81720,11.01157,18.00000,5.25000,0.00000,0.00000,0.18235,0.00000,0.00000,1.00000,0.80392,0.46273
4,7,30,0,1.14286,0.57143,-31.30263,153,6.92667,3.58300,8833342.33333,12.75000,0.00000,1.05000,0.54000,0.00000,0.00000,0.98778,1.04830,0.34534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150441,803273,50,0,17.55556,10.11111,14.73684,214,18.37705,10.79798,1294132.82353,10.52000,0.62766,0.76923,0.61616,0.00000,0.02404,0.52295,0.47475,0.18241
150442,803287,40,1,4.14286,1.42857,-73.00000,579,3.03559,2.47472,4000008.77778,5.42857,0.00000,0.93750,1.63889,0.20833,0.44167,0.87119,0.90000,0.10026
150443,803299,40,0,0.20000,0.10000,-71.39286,122,5.82778,1.30645,5.00000,2.00000,1.00000,0.60909,0.29032,0.26718,0.00000,1.00000,1.00000,0.30091
150444,803305,40,1,2.20000,0.60000,-94.18182,640,12.60526,34.90244,8000018.00000,8.00000,0.67368,0.00000,4.63415,0.14531,0.07500,1.00000,1.00000,0.03871


In [ ]:
pd.set_option("display.max_column", None)
display(CS_DATA_M2_3.loc[(CS_DATA_M2_3["분산투자지표_매수기준"] > 1)])
display(CS_DATA_M3_1.loc[(CS_DATA_M3_1["분산투자지표_매수기준"] > 1)])
display(CS_DATA_M4_2.loc[(CS_DATA_M4_2["분산투자지표_매수기준"] > 1)])
# BEFORE_DATA_M2_3[["특정기간동안의 주식매수액 1위 종목매수액 합계", "특정기간동안의 주식매수액 2위 종목매수액 합계", "특정기간동안의 주식매수액 3위 종목매수액 합계", "특정기간동안의 주식 매수액 합계"]]

___

In [ ]:
class DensityBasedClustering():

  # def __init__(self):
